In [ ]:
import pandas as pd
from bert_serving.client import BertClient

In [1]:
bc = BertClient(ip='localhost', check_length=False)

In [3]:
oc = pd.read_csv('tmp/opencorpora.csv.gz', compression='gzip')
criminal_code = pd.read_csv('tmp/vectors/criminal_code.csv.gz', compression='gzip')
criminal_court_orders = pd.read_csv('tmp/vectors/criminal_court_orders.csv.gz', compression='gzip')
civil_code = pd.read_csv('tmp/vectors/civil_code.csv.gz', compression='gzip')
civil_court_orders = pd.read_csv('tmp/vectors/civil_court_orders.csv.gz', compression='gzip')

oc['source'] = 'opencorpora'
criminal_code['source'] = 'criminal_code'
criminal_court_orders['source'] = 'criminal_court_orders'
civil_code['source'] = 'civil_code'
civil_court_orders['source'] = 'civil_court_orders'

criminal_code['name'] = criminal_code.apply(lambda x: "Ст.{} {}".format(x['article_number'], x['article_name']), axis=1)
civil_code['name'] = civil_code.apply(lambda x: "Ст.{} {}".format(x['article_number'], x['article_name']), axis=1)

civil_court_orders['name'] = ["Гражд. дело {}".format(i) for i in range(civil_court_orders.shape[0])]

In [4]:
tmp = pd.concat([
    oc.reset_index()[['index', 'title', 'text', 'source']].rename(columns={'title':'name'}), 
    criminal_code.reset_index()[['index', 'name', 'article', 'source']].rename(columns={"article": "text"}),
    criminal_court_orders.reset_index()[['index', 'title', 'case',  'source']].rename(columns={'title':'name', "case":"text"}),
    civil_code.reset_index()[['index', 'name', 'article', 'source']].rename(columns={"article": "text"}),
    civil_court_orders.reset_index()[['index', 'name', 'text', 'source']],
]).reset_index(drop=True)

In [13]:
%%time
bert_vec = bc.encode(tmp['text'].tolist())

CPU times: user 4.13 s, sys: 3.28 s, total: 7.41 s
Wall time: 21min 29s


In [22]:
tmp['vectors'] = bert_vec.tolist()

In [27]:
tmp.to_csv("tmp/vectors/bert_all.csv.gz", compression='gzip', index=False)